In [1]:
import torch 
import pandas as pd
import numpy as np

In [2]:
from datasets import load_dataset
dataset = load_dataset("turkish-nlp-suite/turkish-wikiNER")


Found cached dataset turkish-wiki_ner (/home/dsinghvi/.cache/huggingface/datasets/turkish-nlp-suite___turkish-wiki_ner/turkish-WikiNER/1.0.0/8653bd2312d132b985c0abb10bfb847934dfeab8e34ad48ee4502c9d2d455e0f)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
import experiment_runner

In [4]:
self = experiment_runner.ExperimentRunner(cuda=True, seq_len=50, model_name = 'xlm-roberta-base', method=105)

In [5]:
self.prepare_data()

Found cached dataset turkish-wiki_ner (/home/dsinghvi/.cache/huggingface/datasets/turkish-nlp-suite___turkish-wiki_ner/turkish-WikiNER/1.0.0/8653bd2312d132b985c0abb10bfb847934dfeab8e34ad48ee4502c9d2d455e0f)


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
self.prepare_model()

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-st

In [7]:
self.MODEL_NAME

'xlm-roberta-base'

In [9]:
# import random
# def get_prediction_softmax(self, X, token_next):
#     logits = self.model(X).logits
#     if self.SOFTMAX:
#         abc =  logits[0, token_next:, :].softmax(dim=-1)
#     else:
#         abc =  logits[0, token_next:, :]

#     if not self.CUDA:
#         return abc.detach().numpy()
#     else:
#         return abc
    
# def interaction_value_di(self, X, tokens):
#     token1, token2 = tokens
    
#     if self.MODEL_NAME == 'xlm-roberta-base':
#         token_next = 0
#     else:
#         assert False

#     AB = get_prediction_softmax(self, X, token_next)
#     X_t1 = X.clone()
#     X_t1[0, token1] = self.tokenizer.pad_token_id
#     A = self.get_prediction_softmax(X_t1, token_next)

#     X_t2 = X.clone()
#     X_t2[0,token1] = self.tokenizer.pad_token_id
#     B = self.get_prediction_softmax(X_t2, token_next)

#     X_t12 = X.clone()
#     X_t12[0,token2] = self.tokenizer.pad_token_id
#     X_t12[0,token1] = self.tokenizer.pad_token_id
#     phi = self.get_prediction_softmax(X_t12, token_next)

#     # print(AB, A, B, phi)
#     val = AB - A - B + phi


#     if self.METHOD == 105:
#         val = AB - A - B + phi
#         val = torch.divide(torch.linalg.norm(val, dim=1), torch.linalg.norm(AB, dim=1)).cpu()
#         res_list = [(1, val.detach(), token_next)]
#         val = AB - A - B + phi
#         val = torch.divide(torch.linalg.norm(val, dim=1), torch.linalg.norm(AB - phi, dim=1)).cpu()
#         res_list.append((2, val.detach(), token_next))
#         val = AB - A - B + phi
#         val = torch.linalg.norm(val, dim=1).cpu()
#         res_list.append((3, val.detach(), token_next))
#         val = torch.divide(torch.linalg.norm(AB - A - B, dim=1), torch.linalg.norm(AB, dim=1)).cpu()
#         res_list.append((4, val.detach(), token_next))
#         return res_list

#     assert False

#     if self.METHOD == 1:
#         val = torch.divide(torch.linalg.norm(val, dim=1), torch.linalg.norm(AB, dim=1)).cpu()
#         return val.detach(), token_next
#     elif self.METHOD == 2:
#         val = torch.divide(torch.linalg.norm(val, dim=1), torch.linalg.norm(AB - phi, dim=1)).cpu()
#         return val.detach(), token_next
#     elif self.METHOD == 3:
#         val = torch.linalg.norm(val, dim=1).cpu()
#         return val.detach(), token_next
#     elif self.method == 4:
#         val = torch.divide(torch.linalg.norm(AB - A - B, dim=1), torch.linalg.norm(AB, dim=1)).cpu()
#         return val.detach(), token_next



# def calculate_interaction(self, encoded_row, row_number):
#     interactions = []
#     encoded_len = len(encoded_row)
#     for j in range( min(self.SEQ_LEN, encoded_len)):
#         probability = 0.05
#         if random.random() < probability: 
#             for k in range(j+1, min(self.SEQ_LEN, encoded_len, j+9)):
#                 if j+k >= encoded_len:
#                     continue
#                 if encoded_row[j] == self.tokenizer.unk_token_id or encoded_row[k] == self.tokenizer.unk_token_id:
#                     continue

#                 og = encoded_row.clone()
#                 og = og.reshape(1, -1)
#                 iv = interaction_value_di(self, og, [j, k])
#                 interactions.append([iv, abs((j-k)), row_number, j, k])

#     return interactions

import tqdm.auto as tqdm
for row_number, row in tqdm.tqdm(enumerate(self.test), total=len(self.test)):
    average_distance = []
    encoded_row =  self.tokenizer(row, padding=False, is_split_into_words=True, truncation=True, max_length=self.SEQ_LEN, return_tensors ='pt')
    g = {}
    for ix, el in enumerate(encoded_row.word_ids()):
        if el is not None:
            if el not in g:
                g[el] = []
            g[el].append(ix)
    mwe = g.values()
    encoded_row = encoded_row.input_ids[0]
    if self.CUDA:
        encoded_row = encoded_row.cuda()

    average_distance.extend(self.calculate_interaction(encoded_row, row_number))
        
    if row_number % 1000 == 0:
        print(len(average_distance))
        pickle.dump(average_distance, open(f'avg_{self.MODEL_NAME}{suffix}.pkl','wb'))

  0%|          | 0/2000 [00:00<?, ?it/s]

8


NameError: name 'pickle' is not defined

In [ ]:
# dataset = load_dataset("turkish-nlp-suite/Corona-mini")
# dataset

In [ ]:

# np.sum([ len(dataset['train']['text'][i].split('.')) for i in range(len(dataset['train']['text']))])